# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [1]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [2]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analysis')
results

,Title,Company,Location
0,"Data Analytics Associate, CrossInstall",General Mills,"San Francisco, CA"
1,DATA SCIENTIST I,The Home Depot,"Houston, TX"
2,Junior Data Scientist Apprenticeship,IBM,"New York, NY"
3,Data Science - Intern,"Sonde Health, Inc.","Boston, MA"
4,Data Science Intern,Curology,"San Francisco, CA"
5,Data Analyst - Chicago,SpiderRock,"Chicago, IL"
6,Financial Analytics Consultant,Toyota North America,"Dallas, TX"
7,Research Assistant IV Non-Lab (Research Data A...,Harvard University,"Boston, MA"
8,"Analyst, Data and Analysis",Digitas India,"Boston, MA"
9,Data Analyst,"CapTech Ventures, Inc","Charlotte, NC"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [20]:
# your code here

def scrape_linkedin_job_search(keywords, num_page):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    
    # Assemble the full url with parameters
    scrape_url = ([''.join([BASE_URL, 'keywords=', keywords, "&start=", str(25 *i)]) 
                  for i in range (num_page)])
    soup = []
    
    for url in scrape_url:
        if requests.get(url).status_code == 200:
            soup.append(BeautifulSoup(requests.get(url).text, 'html.parser'))
        else:
            break
    
    # Create a request to get the data from the server 

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for p in soup:
        for card in p.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

results2 = scrape_linkedin_job_search('data%20analysis', 5)
results2

,Title,Company,Location
0,Business Analyst Level-1,Staffigo,"Santa Clara, CA"
1,Business Analyst Level-1,Staffigo,"Newark, NJ"
2,Business Analyst,Staffigo,"Stamford, CT"
3,Business Analyst Fresher,Staffigo,"Portland, OR"
4,Data Warehouse Architect,Kent Corporation,"Davenport, IA"
...,...,...,...
120,Data Analyst - Remote Position! - Dallas,Seasoned Recruitment,"Dallas, TX"
121,Quantitative Trading Analyst Intern,DRW,"Chicago, IL"
122,Data Analyst I,Massachusetts General Hospital,"Charlestown, NH"
123,Data Analyst,Booz Allen Hamilton,"Arlington, VA"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [25]:
# your code here

def scrape_linkedin_job_search(keywords, location, num_page ):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    
    # Assemble the full url with parameters
    scrape_url = ([''.join([BASE_URL, 'keywords=', keywords, "&location=", location, "&start=", str( 25 *i)])
                   for i in range (num_page)])
    soup = []
    
    for url in scrape_url:
        if requests.get(url).status_code == 200:
            soup.append(BeautifulSoup(requests.get(url).text, 'html.parser'))
        else:
            break
    
    # Create a request to get the data from the server 

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for p in soup:
        for card in p.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

results3 = scrape_linkedin_job_search('data%20analysis', 'Germany', 1)
results3


,Title,Company,Location
0,(Junior) Data Scientist / Data Analyst (w/m/d),KPMG Deutschland,"Dresden, Saxony, Germany"
1,Data Science Intern,datalytix.io,"Stuttgart, Baden-Württemberg, Germany"
2,Data Analyst,Human Capital Advisory Group,Stuttgart Region
3,Data Analyst (m/f/x),"Audible, Inc.","Berlin, Berlin, Germany"
4,Data Scientist (m/f/d),Bayer,"Weimar, Thuringia, Germany"
5,Internship in Data Analytics,Henkel,"Düsseldorf, North Rhine-Westphalia, Germany"
6,Data Scientist / Data Analyst (m/f/d),Chemovator GmbH,"Mannheim, Baden-Württemberg, Germany"
7,Data Intern,Pandata,"Berlin, Berlin, Germany"
8,Data Analyst (f/m/x),audibene,"Berlin, Berlin, Germany"
9,(Junior) Data Analyst - Marketing,Trade Republic,"Berlin, Berlin, Germany"


## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [26]:
# your code here
def scrape_linkedin_job_search(keywords, location, num_page, num_days ):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    
    # Assemble the full url with parameters
    scrape_url = ([''.join([BASE_URL, "f_TPR=r", str(num_days * 86400), 'keywords=', keywords, "&location=", location, "&start=", str( 25 *i)])
                   for i in range (num_page)])
    soup = []
    
    for url in scrape_url:
        if requests.get(url).status_code == 200:
            soup.append(BeautifulSoup(requests.get(url).text, 'html.parser'))
        else:
            break
    
    # Create a request to get the data from the server 

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for p in soup:
        for card in p.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

results4 = scrape_linkedin_job_search('data%20analysis', 'Germany', 1, 3)
results4



,Title,Company,Location
0,Intern Commercial Planning Intern (M/F/D),Zalando SE,"Berlin, Berlin, Germany"
1,Remote Data Entry Clerk - Work at Home,WW-Recruit,"Hamburg, Germany"
2,Managing Director,ACL Partners,"Frankfurt, Hesse, Germany"
3,Director Business Innovation Consulting (all g...,Zühlke Group,Stuttgart Region
4,Country Manager,Charlton Morris,"Munich, Bavaria, Germany"
5,Expansion Projects Manager (m/f/d),TIER Mobility,"Berlin, Berlin, Germany"
6,Global Capital Markets - Head of Private Side ...,Morgan Stanley,Frankfurt Rhine-Main Metropolitan Area
7,Marketing Analyst (m/f/d),FREE NOW (formerly mytaxi),"Hamburg, Hamburg, Germany"
8,Internships,WWP,"Berlin, Berlin, Germany"
9,Junior Projekt Manager Luftfahrt (m/w/d),AIP SERVICES GmbH,"Munich, Bavaria, Germany"


## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.

In [ ]:
# your code here
def scrape_linkedin_job_search(keywords, location, num_page, num_days ):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
   currentJobId=2006865666 
    # Assemble the full url with parameters
    scrape_url = ([''.join([BASE_URL, "f_TPR=r", str(num_days * 86400), 'keywords=', keywords, "&location=", location, "&start=", str( 25 *i)])
                   for i in range (num_page)])
    soup = []
    
    for url in scrape_url:
        if requests.get(url).status_code == 200:
            soup.append(BeautifulSoup(requests.get(url).text, 'html.parser'))
        else:
            break
    
    # Create a request to get the data from the server 

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for p in soup:
        for card in p.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            for item in card:
                seniority_level = card.findChild("h3", )
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

results5 = scrape_linkedin_job_search('data%20analysis', 'Germany', 1, 3, )
results5
